In [8]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [10]:
cap = cv.VideoCapture('project_video.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame
    cv.imshow('Frame',frame)
    
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()